In [1]:
import agate

In [2]:
commercials = agate.Table.from_csv('../../Data/commercials.csv')

In [3]:
print(commercials)

|----------------+---------------|
|  column_names  | column_types  |
|----------------+---------------|
|  wp_identifier | Number        |
|  network       | Text          |
|  market        | Text          |
|  location      | Text          |
|  program       | Text          |
|  program_type  | Text          |
|  start_time    | Date          |
|  end_time      | Date          |
|  archive_id    | Text          |
|  embed_url     | Text          |
|  sponsor       | Text          |
|  sponsor_type  | Text          |
|  subject       | Text          |
|  candidate     | Text          |
|  type          | Text          |
|  message       | Text          |
|  air_count     | Number        |
|  market_count  | Number        |
|  date_created  | Date          |
|----------------+---------------|



In [16]:
commercials_with_months = commercials.compute([
    ('month', agate.Formula(agate.Number(), lambda row: '%s' % row['start_time'].month))
])

In [17]:
by_market = commercials_with_months.group_by('market')

In [18]:
by_market_by_month = by_market.group_by('month')

In [19]:
market_month_totals = by_market_by_month.aggregate([
    ('count', agate.Length())
])

In [20]:
market_month_totals = market_month_totals.order_by(lambda row: (row['market'], row['month']))

In [24]:
market_month_totals.print_table()

|---------+-------+---------|
|  market | month |  count  |
|---------+-------+---------|
|  BOS    |     1 |  9,820  |
|  BOS    |     2 |  4,655  |
|  BOS    |     3 |     67  |
|  BOS    |    10 |      5  |
|  BOS    |    11 |    589  |
|  BOS    |    12 |  5,847  |
|  CAE    |     1 |  2,056  |
|  CAE    |     2 |  7,499  |
|  CAE    |    12 |    109  |
|  CID    |     1 | 10,820  |
|  CID    |     2 |    611  |
|  CID    |    10 |      4  |
|  CID    |    11 |    471  |
|  CID    |    12 |  3,848  |
|  CLE    |     1 |    246  |
|  CLE    |     2 |    287  |
|  CLE    |     3 |      7  |
|  CLT    |     2 |    134  |
|  COS    |     1 |     15  |
|  COS    |     2 |  1,088  |
|  COS    |     3 |     70  |
|  CVG    |     1 |    197  |
|  CVG    |     2 |    344  |
|  CVG    |     3 |     20  |
|  DEN    |     1 |     28  |
|  DEN    |     2 |  1,484  |
|  DEN    |     3 |     62  |
|  DSM    |     1 | 10,890  |
|  DSM    |     2 |    707  |
|  DSM    |     3 |      1  |
|  DSM    